##Comentarios
En general, las aproximaciones realizadas son adecuadas, pero se podría mejorar el enfoque en dos aspectos principales:

Aunque se empleó la validación cruzada para evaluar el rendimiento del modelo, se omitió reservar un conjunto de datos específico para pruebas. Esto es esencial para obtener una evaluación imparcial del modelo final.

Además, el análisis se centró únicamente en la marca del vehículo, sin tomar en cuenta el modelo específico, lo cual era un requisito del ejercicio.

Tras mi revisión, concluyo que el mejor modelo para este caso es una regresión lineal regularizada con un valor de λ alto . Este modelo alcanzó un RMSE de aproximadamente 1656


## puntaje 15/20



# Parte 3 carros

Para este ejercicio usaremos el dataset LondonCars2014, el cual contiene información de varios carros, en la que esta incluida el año de fabricación, su kilometraje y su precio, que son las variables que nos interesan para este ejercicio. En este caso, intentaremos predecir el precio de una marca de auto en concreto en función de su año de fabricación y su kilometraje.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('LondonCars2014.csv')
data

,Make,Model,Year,Mileage,Price,Body Style,Ex Color,In Color,Engine,Transmission,Doors
0,Toyota,Avalon XLE,2014,4725,28995,Sedan,Gray,Black,6 Cyl,Automatic,4
1,Ford,Escape SE,2014,17201,24994,SUV,Silver,Black,4 Cyl,Automatic,4
2,Hyundai,Santa Fe Sport,2014,6279,25998,SUV,Silver,Gray,4 Cyl,Automatic,4
3,Ford,Escape SE,2014,16262,26684,SUV,Black,Black,4 Cyl,Automatic,4
4,Jeep,Grand Cherokee Limited Edition,2014,35572,33440,SUV,Black,Black,6 Cyl,Automatic,4
...,...,...,...,...,...,...,...,...,...,...,...
9075,Chevrolet,Malibu LT,2010,40340,12998,Sedan,Silver,Black,4 Cyl,Automatic,4
9076,Ford,Edge SEL,2009,43169,18495,SUV,Blue,Gray,6 Cyl,Automatic,4
9077,Dodge,Avenger SXT,2012,6761,14944,Sedan,Silver,Black,4 Cyl,Automatic,4
9078,Mercedes-Benz,ML 350,2011,38215,35900,SUV,Silver,Black,6 Cyl,Automatic,4


En este conjunto de datos podemos ver que hay una gran cantidad de variables categóricas las cuales son:

- Make: La marca del fabricante

- Model: El modelo del automóvil a vender

- Body Style: El tipo de carro que se está vendiento

- Ex Color: El color del exterior

- In Color: El clolor del interior

- Engine: El tipo de motor

- Transmission: El tipo de transmisión

- Doors: El número de puertas que puede ser 2 o 4

Y 3 variables numéricas las cuales son:

- Year: El año de fabricación

- Mileage: El kilometraje del carro

- Price: El precio de venta del carro

Que son las variables que se utilizarán

Escogeremos la marca de autos que tenga un mayor número de observaciones para este ejercicio

In [ ]:
data.groupby('Make').count()['Model'].sort_values(ascending=False)

Make
Nissan           1035
Honda             917
Toyota            855
Mercedes-Benz     691
Ford              659
BMW               612
Lexus             454
Acura             393
Chevrolet         350
Jeep              321
Hyundai           312
Infiniti          302
Audi              231
Volkswagen        213
Cadillac          199
Dodge             173
Subaru            162
Volvo             126
Mazda             124
Chrysler          119
GMC               116
Lincoln            98
Kia                96
Land Rover         86
Porsche            76
Mini               52
Mitsubishi         50
Buick              47
Ram                26
Pontiac            26
Jaguar             26
Mercury            26
Scion              22
Saturn             19
Saab               17
Hummer             11
Ferrari             7
Suzuki              5
Aston               4
Lamborghini         4
Fiat                3
Rolls-Royce         3
Plymouth            2
Maybach             2
Maserati            2
Smart

En este sentido nos quedaremos con los autos de marca Nissan

In [ ]:
df = data.loc[data['Make'] == 'Nissan']
df = df[['Year', 'Mileage', 'Price']]
df

,Year,Mileage,Price
9,2014,20992,34999
24,2013,29483,17768
41,2013,5457,17777
53,2013,15909,16530
74,2013,29886,19995
...,...,...,...
9055,2008,78308,16995
9060,2011,28552,18995
9069,2011,36693,23277
9072,2008,104373,13450


A continuación visualizaremos los datos, primero en 3 dimensiones

In [ ]:
fig = px.scatter_3d(df, x='Year', y='Mileage', z='Price')
fig.show()

Y veremos plots de los datos 2 a 2

In [ ]:
fig = px.scatter(x=df['Year'], y=df['Price'])
fig.update_layout(
    title = 'Year vs Price',
    xaxis_title="Year",
    yaxis_title="Price",
)
fig.show()

In [ ]:
fig = px.scatter(x=df['Mileage'], y=df['Price'])
fig.update_layout(
    title = 'Mileage vs Price',
    xaxis_title="Mileage",
    yaxis_title="Price",
)
fig.show()

Podemos ver que hay una clara tendencia a que los carros más nuevos y los carros que tengan menor kilometraje son aquellos carros que se venderán a un mayor precio. A continuación buscaremos un modelo de regresión lineal que ajuste los datos.

# Búsqueda del mejor modelo

Para buscar el mejor modelo en primera instancia utilizaremos cross validation, dividiremos el conjunto en 10 folds y buscaremos el mejor grado para realizar una regresión lineal sin regularización

In [ ]:
X = df[['Year', 'Mileage']]
y = df['Price']

errors = []
for i in range(1,7):
    poly = PolynomialFeatures(i)
    X_pol = poly.fit_transform(X)
    col =poly.get_feature_names_out().tolist()
    X_pol = pd.DataFrame(X_pol, columns=col)
    X_pol = X_pol.drop(['1'], axis=1)
    lm = LinearRegression()
    mse_score = cross_val_score(lm, X_pol, y, cv=10, scoring = 'neg_root_mean_squared_error')
    errors.append(-np.mean(mse_score))
    print(f'Para un modelo de grado {i} se obtiene un RMSE promedio de: {np.mean(mse_score)} y una varianza de: {np.var(mse_score)}')

fig = px.line(x=range(1,7), y=errors)
fig.update_layout(
    title = 'Error de modelos polinomiales',
    xaxis_title="Grado del polinomio",
    yaxis_title="RMSE",
)
fig.show()

Para un modelo de grado 1 se obtiene un RMSE promedio de: -4840.816724002804 y una varianza de: 1270395.387479318
Para un modelo de grado 2 se obtiene un RMSE promedio de: -4817.339186021615 y una varianza de: 1238210.056516781
Para un modelo de grado 3 se obtiene un RMSE promedio de: -4817.19400472457 y una varianza de: 1218842.228643554
Para un modelo de grado 4 se obtiene un RMSE promedio de: -4821.145800377175 y una varianza de: 1178553.8961381651
Para un modelo de grado 5 se obtiene un RMSE promedio de: -4831.491211315528 y una varianza de: 1181741.5474020736
Para un modelo de grado 6 se obtiene un RMSE promedio de: -5041.629699811544 y una varianza de: 1526128.9604288284


Con los resultados obtenidos por la validación cruzada, podemos ver que el modelo que mejor se ajusta a los datos sería un modelo de grado 3, pues este minimiza el error cuadrático medio de la raíz (RMSE) y no tiene una varianza alta respecto a los modelos de otros grados, además, al aumentar a grado 5, este error aumenta significativamente. Veremos el resultado de esta regresión:

In [ ]:
X = df[['Year', 'Mileage']]
y = df['Price']
poly = PolynomialFeatures(3)
X_pol = poly.fit_transform(X)
col =poly.get_feature_names_out().tolist()
X_pol = pd.DataFrame(X_pol, columns=col)
X_pol = X_pol.drop(['1'], axis=1)
best_lin = LinearRegression()
best_lin.fit(X_pol,y)
Xplot = pd.DataFrame()
n = 100
Xplot['Year'] = np.linspace(1998,2014,n)
Xplot['Mileage'] = np.linspace(0,200000,n)
zplot = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        x_pred = np.array([Xplot['Year'][i], Xplot['Mileage'][j]]).reshape(1,-1)

        x_pred= poly.fit_transform(x_pred)
        X_pred = pd.DataFrame(x_pred, columns=col)
        X_pred = X_pred.drop(['1'], axis=1)
        zplot[i][j] = best_lin.predict(X_pred)

fig = go.Figure(data=[go.Surface(z=zplot,x=Xplot['Year'], y=Xplot['Mileage']),
                        go.Scatter3d(x=df['Year'], y=df['Mileage'], z=df['Price'], mode='markers')])
fig.update_layout(
    title = 'Regresión de grado 3',
)
fig.show()

A continuación, veamos si un modelo con regularización Ridge o Lasso puede obtener mejores resultados que el modelo anterior

In [ ]:
alfas = {'alpha': np.linspace(0,2, 1000)}

GS_R_DF = pd.DataFrame()
errors = []
for i in range(1,6):
    poly = PolynomialFeatures(i)
    X_pol = poly.fit_transform(X)
    col =poly.get_feature_names_out().tolist()
    X_pol = pd.DataFrame(X_pol, columns=col)
    X_pol = X_pol.drop(['1'], axis=1)
    lm_R = Ridge()
    GS_R = GridSearchCV(lm_R, alfas, cv=10, scoring = 'neg_root_mean_squared_error')
    GS_R.fit(X_pol, y)
    deg_res =  pd.DataFrame.from_dict(GS_R.cv_results_)
    deg_res['degree'] = [i for j in range(len(deg_res))]
    GS_R_DF = pd.concat([GS_R_DF, deg_res])
    besto_index = GS_R.best_index_
    print(f"Para un modelo de grado {i} el mejor alfa es {deg_res['param_alpha'][besto_index]} con un mse promedio de: {deg_res['mean_test_score'][besto_index]} y un error estandar: {deg_res['std_test_score'][besto_index]}")
    errors.append(-deg_res['mean_test_score'][besto_index])

fig = px.line(x=range(1,6), y=errors)
fig.update_layout(
    title = 'Error de modelos ridge',
    xaxis_title="Grado del polinomio",
    yaxis_title="RMSE",
)
fig.show()
GS_R_DF = GS_R_DF[['param_alpha', 'mean_test_score', 'std_test_score', 'degree']]
GS_R_DF.sort_values('mean_test_score', ascending=False, inplace=True)
GS_R_DF.head(5)

Para un modelo de grado 1 el mejor alfa es 2.0 con un mse promedio de: -4840.812057997735 y un error estandar: 1127.1244284541272
Para un modelo de grado 2 el mejor alfa es 0.0 con un mse promedio de: -4817.339188018635 y un error estandar: 1112.7488738682503
Para un modelo de grado 3 el mejor alfa es 0.07807807807807808 con un mse promedio de: -4808.3801380610985 y un error estandar: 1105.9889743867273
Para un modelo de grado 4 el mejor alfa es 0.8368368368368369 con un mse promedio de: -4792.483328750399 y un error estandar: 1088.9158760240484
Para un modelo de grado 5 el mejor alfa es 0.6386386386386387 con un mse promedio de: -9366.16917190432 y un error estandar: 6581.790028811212


,param_alpha,mean_test_score,std_test_score,degree
418,0.836837,-4792.483329,1088.915876,4
384,0.768769,-4792.486951,1088.920426,4
840,1.681682,-4792.490536,1088.932522,4
904,1.80981,-4792.499814,1088.948653,4
620,1.241241,-4792.501017,1088.901399,4


In [ ]:
alfas = {'alpha': np.linspace(0.01,2, 1000)}

GS_L_DF = pd.DataFrame()
for i in range(1,6):
    poly = PolynomialFeatures(i)
    X_pol = poly.fit_transform(X)
    col =poly.get_feature_names_out().tolist()
    X_pol = pd.DataFrame(X_pol, columns=col)
    X_pol = X_pol.drop(['1'], axis=1)
    lm_L = Lasso()
    GS_L = GridSearchCV(lm_L, alfas, cv=10, scoring = 'neg_root_mean_squared_error')
    GS_L.fit(X_pol, y)
    deg_res =  pd.DataFrame.from_dict(GS_L.cv_results_)
    deg_res['degree'] = [i for j in range(len(deg_res))]
    GS_L_DF = pd.concat([GS_L_DF, deg_res])
    besto_index = GS_L.best_index_
    print(f"Para un modelo de grado {i} el mejor alfa es {deg_res['param_alpha'][besto_index]} con un mse promedio de: {deg_res['mean_test_score'][besto_index]} y un error estandar: {deg_res['std_test_score'][besto_index]}")

GS_L_DF = GS_L_DF[['param_alpha', 'mean_test_score', 'std_test_score', 'degree']]
GS_L_DF.sort_values('mean_test_score', ascending=False, inplace=True)
GS_L_DF.head(5)

Para un modelo de grado 1 el mejor alfa es 0.06378378378378378 con un mse promedio de: -4840.816089067913 y un error estandar: 1127.1185930793883
Para un modelo de grado 2 el mejor alfa es 0.01 con un mse promedio de: -4831.784654289373 y un error estandar: 1113.7209502551864
Para un modelo de grado 3 el mejor alfa es 0.01 con un mse promedio de: -4831.864758972492 y un error estandar: 1113.3273151101357
Para un modelo de grado 4 el mejor alfa es 0.01 con un mse promedio de: -4833.112450624512 y un error estandar: 1114.2081167334793
Para un modelo de grado 5 el mejor alfa es 0.01 con un mse promedio de: -4835.739614935387 y un error estandar: 1113.9399925882594


,param_alpha,mean_test_score,std_test_score,degree
0,0.01,-4831.784654,1113.720950,2
1,0.011992,-4831.784678,1113.720958,2
2,0.013984,-4831.784701,1113.720966,2
3,0.015976,-4831.784724,1113.720973,2
4,0.017968,-4831.784748,1113.720981,2


Al igual que en la parte 2 , podemos ver que el modelo con regularización tipo Lasso no presenta mejoras significativas con respecto al modelo original, sin embargo, en este caso la regularización de tipo Ridge si mejora los resultados, con lo cual el mejor modelo de regresión resulta ser un modelo de regularización tipo Ridge de grado 4, que mostraremos a continuación

In [ ]:
X = df[['Year', 'Mileage']]
y = df['Price']
poly = PolynomialFeatures(4)
X_pol = poly.fit_transform(X)
col =poly.get_feature_names_out().tolist()
X_pol = pd.DataFrame(X_pol, columns=col)
X_pol = X_pol.drop(['1'], axis=1)
best_ridge = Ridge(alpha=1.2412412412)
best_ridge.fit(X_pol,y)
Xplot = pd.DataFrame()
n = 100
Xplot['Year'] = np.linspace(1998,2014,n)
Xplot['Mileage'] = np.linspace(0,200000,n)
zplot = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        x_pred = np.array([Xplot['Year'][i], Xplot['Mileage'][j]]).reshape(1,-1)

        x_pred= poly.fit_transform(x_pred)
        X_pred = pd.DataFrame(x_pred, columns=col)
        X_pred = X_pred.drop(['1'], axis=1)
        zplot[i][j] = best_ridge.predict(X_pred)

fig = go.Figure(data=[go.Surface(z=zplot,x=Xplot['Year'], y=Xplot['Mileage']),
                        go.Scatter3d(x=df['Year'], y=df['Mileage'], z=df['Price'], mode='markers')])
fig.update_layout(
    title = 'Regresión Ridge de grado 4',
)
fig.show()

# Conclusión

Tras realizar procesos de validación cruzada, llegamos a que el mejor modelo es un modelo de grado 3 que toma las variables Year y Mileage con los siguientes coeficientes

In [ ]:
best_ridge.coef_

array([-1.06848346e-01,  9.80657487e+00, -1.42243950e+02,  1.61630003e+00,
        4.38270501e-02,  1.87495495e-01, -1.54863647e-03, -4.48867749e-05,
        6.34169095e-09, -5.22328328e-05,  3.69229790e-07,  1.14819026e-08,
       -3.12662630e-12, -1.42138190e-16])

Con intercepto:

In [ ]:
best_ridge.intercept_

-95316427.80688561

Y con un alpha de regularización de

In [ ]:
best_ridge.alpha

1.2412412412

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4d95ea4b-1989-49a0-85a4-49dea6db2700' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>